[![Run Jupyter Notebooks](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/RichardPotthoff/Notebooks/main?filepath=Sync2way.ipynb)   <- click here to open this file in MyBinder
    
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RichardPotthoff/Notebooks/blob/main/Sync2way.ipynb)   <- click here to open this file in Google Colab

In [1]:
def capture_console(code):
    return (
"""
try{console._o=console._o||{log:console.log,error:console.error,warn:console.warn,info:console.info};[['log'],['error','red'],['warn'],['info']].forEach(([t,c="black"])=>{console[t]=console._o[t];let d=element.consoleOutput;if(!d){d=document.createElement("div");d.id="console-output";d.style.cssText="white-space:pre-wrap;font-family:monospace;padding:0px;background:#f0f0f0;line-height:1.1;";element.appendChild(d);element.consoleOutput=d;}let o=console[t],n=o;while(n&&n.toString().indexOf("[native code]")<0)n=n.apply?function(...a){return n.apply(this,a);}:null;o=function(...a){(n||console._o[t]).apply(console,a);let s=a.map(e=>typeof e==='object'?JSON.stringify(e,null,2):e).join(' ');const m=document.createElement("div");m.innerText=`[${t.toUpperCase()}] ${s}`;m.style.cssText=`margin:0;line-height:1.1;padding:0px 0;color:${c};`;d.appendChild(m);};console[t]=o;});
"""  +
code + 
"""
}catch(e){console.error("Error:",e);}finally{[['log'],['error','red'],['warn'],['info']].forEach(([t])=>console._o&&console._o[t]&&(console[t]=console._o[t]));}
"""
)

## 2-way synchronisation using text input ipywidgets  

In [2]:
import ipywidgets as widgets
from IPython.display import display, Javascript
import uuid

# Create unique UUIDs for the widgets
slider_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')
text_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')
hidden_text_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')

# Create the slider, text, and hidden text widgets with unique classes
slider = widgets.IntSlider(value=50, min=0, max=100)
slider.add_class(slider_uuid)
text_widget = widgets.Text(value="initial text")
text_widget.add_class(text_uuid)
hidden_text = widgets.Text(value="50", layout={'display': 'none'})
hidden_text.add_class(hidden_text_uuid)

# Two-way linking
slider.observe(lambda change: setattr(hidden_text, 'value', str(change['new'])), names='value')
hidden_text.observe(lambda change: setattr(slider, 'value', int(change['new'])), names='value')

# Create the Output widget with custom inputs
output1 = widgets.Output()
with output1:
    display(Javascript(capture_console(f"""
    const sliderInput = document.createElement('input');
    sliderInput.type = 'range';
    sliderInput.min = 0;
    sliderInput.max = 100;
    sliderInput.value = 50;
    sliderInput.style.width = '210px';
    element.appendChild(sliderInput);

    const textInput = document.createElement('input');
    textInput.type = 'text';
    textInput.value = 'initial text';
    textInput.style.width = '200px';
    element.appendChild(textInput);

    // JavaScript-to-Python synchronization
    const sliderWidget = document.querySelector('.{slider_uuid}');
    const hiddenTextWidget = document.querySelector('.{hidden_text_uuid}');
    const textWidget = document.querySelector('.{text_uuid}');

    sliderInput.addEventListener('input', () => {{
        if (hiddenTextWidget) {{
            const input = hiddenTextWidget.querySelector('input');
            if (input) {{
                input.value = sliderInput.value;
                const changeEvent = new Event('change', {{ bubbles: true }});
                input.dispatchEvent(changeEvent);
                const inputEvent = new Event('input', {{ bubbles: true }});
                input.dispatchEvent(inputEvent);
            }}
        }}
    }});

    textInput.addEventListener('input', () => {{
        if (textWidget) {{
            const input = textWidget.querySelector('input');
            if (input) {{
                input.value = textInput.value;
                const changeEvent = new Event('change', {{ bubbles: true }});
                input.dispatchEvent(changeEvent);
                const inputEvent = new Event('input', {{ bubbles: true }});
                input.dispatchEvent(inputEvent);
            }}
        }}
    }});

    // Python-to-JavaScript synchronization (polling)
    setInterval(() => {{
    try{{console._o=console._o||{{log:console.log,error:console.error,warn:console.warn,info:console.info}};[['log'],['error','red'],['warn'],['info']].forEach(([t,c="black"])=>{{console[t]=console._o[t];let d=element.consoleOutput;if(!d){{d=document.createElement("div");d.id="console-output";d.style.cssText="white-space:pre-wrap;font-family:monospace;padding:0px;background:#f0f0f0;line-height:1.1;";element.appendChild(d);element.consoleOutput=d;}}let o=console[t],n=o;while(n&&n.toString().indexOf("[native code]")<0)n=n.apply?function(...a){{return n.apply(this,a);}}:null;o=function(...a){{(n||console._o[t]).apply(console,a);let s=a.map(e=>typeof e==='object'?JSON.stringify(e,null,2):e).join(' ');const m=document.createElement("div");m.innerText=`[${{t.toUpperCase()}}] ${{s}}`;m.style.cssText=`margin:0;line-height:1.1;padding:0px 0;color:${{c}};`;d.appendChild(m);}};console[t]=o;}});
  //       console.log("in set_imtervall");
        if (sliderWidget) {{
            const readout = sliderWidget.querySelector('.widget-readout');
            if (readout && sliderInput.value !== readout.textContent) {{
                sliderInput.value = readout.textContent;
            }}
        }}
        if (textWidget) {{
            const input = textWidget.querySelector('input');
            if (input && textInput.value !== input.value) {{
                textInput.value = input.value;
            }}
        }}
    }}catch(e){{console.error("Error:",e);}}finally{{[['log'],['error','red'],['warn'],['info']].forEach(([t])=>console._o&&console._o[t]&&(console[t]=console._o[t]));}}
    }}, 100); // Poll every 100ms
    """)))

# Display the widgets
display(slider, text_widget, hidden_text, output1)

IntSlider(value=50, _dom_classes=('uuidb5ffc6f6f2e84471817b95a954954239',))

Text(value='initial text', _dom_classes=('uuid860df0d4dd0f4c3bbc76ed2218222bf3',))

Text(value='50', layout=Layout(display='none'), _dom_classes=('uuid94ea6856282244ab9e2621f34c24a815',))

Output()

In [3]:
display(slider, text_widget, hidden_text, output1)

IntSlider(value=50, _dom_classes=('uuidb5ffc6f6f2e84471817b95a954954239',))

Text(value='initial text', _dom_classes=('uuid860df0d4dd0f4c3bbc76ed2218222bf3',))

Text(value='50', layout=Layout(display='none'), _dom_classes=('uuid94ea6856282244ab9e2621f34c24a815',))

Output()